In [ ]:
import pandas as pd
ranking_formatted_statements_df = pd.read_csv('ranking_formatted_statements.csv')
query_pair_formatted_statements_df = pd.read_csv('query_pair_formatted_statements.csv')
sentiment_analysis_df = pd.read_csv('Sentiment_Analysis_df.csv')

In [ ]:
ranking_formatted_statements_df

,0
0,A user has made a query with keyword 'model m ...
1,A user has made a query with keyword 'theprice...
2,A user has made a query with keyword 'accesori...
3,A user has made a query with keyword 'bulk kin...
4,A user has made a query with keyword 'skin tig...
...,...
50927,You are an intelligent shopping assistant that...
50928,You are an intelligent shopping assistant that...
50929,You are an intelligent shopping assistant that...
50930,You are an intelligent shopping assistant that...


In [ ]:
query_pair_formatted_statements_df

,0
0,Which of the following statements best describ...
1,Which of the following statements best describ...
2,Which of the following statements best describ...
3,Which of the following statements best describ...
4,Which of the following statements best describ...
...,...
35995,Which of the following statements best describ...
35996,Which of the following statements best describ...
35997,Which of the following statements best describ...
35998,Which of the following statements best describ...


In [ ]:
sentiment_analysis_df

,input_field,output_field
0,Instructions: Evaluate the following product r...,5
1,Instructions: Evaluate the following product r...,5
2,Instructions: Evaluate the following product r...,1
3,Instructions: Evaluate the following product r...,4
4,Instructions: Evaluate the following product r...,5
...,...,...
29995,Instructions: Evaluate the following product r...,2
29996,Instructions: Evaluate the following product r...,5
29997,Instructions: Evaluate the following product r...,5
29998,Instructions: Evaluate the following product r...,1


In [ ]:
query_pair_formatted_statements_list = query_pair_formatted_statements_df['0'].tolist()
query_pair_formatted_statements_list[:5]

['Which of the following statements best describes the relation from query "15 microsoft card" to query "15 ml containers with lids"? Just reply with a number from 0 to 3, no more analyze.\n0. narrowing\n1. substitute\n2. irrelevant\n3. complement\nAnswer: ',
 'Which of the following statements best describes the relation from query "a history of western society 13th edition" to query "a house is not a home without a dog"? Just reply with a number from 0 to 3, no more analyze.\n0. narrowing\n1. substitute\n2. irrelevant\n3. complement\nAnswer: ',
 'Which of the following statements best describes the relation from query "5.56 dummy rounds" to query "5.5in drawer pull"? Just reply with a number from 0 to 3, no more analyze.\n0. narrowing\n1. substitute\n2. irrelevant\n3. complement\nAnswer: ',
 'Which of the following statements best describes the relation from query "1/2 cc syringe without needle" to query "1/2 copper coupling without stop"? Just reply with a number from 0 to 3, no mor

In [ ]:
from openai import OpenAI

YOUR_KEY = "sk-KseTKIJtYF0BKQZG05835862F82f4c3989CcA2De09F31510"
YOUR_BASE_URL = "https://one-api.qkvlab.com/v1"
YOUR_MODEL = "claude-3-sonnet-20240229"

client = OpenAI(api_key=YOUR_KEY)
client.base_url = YOUR_BASE_URL

completion = client.chat.completions.create(
    model=YOUR_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions."},
        {"role": "user", "content": "A user has made a query with keyword 'model m mechanical keyboard'. Given the following numbered list of 5 products, please rank the products according their relevance with the query. \nProduct List: \n1. Unicomp Two Tone Ultra Classic Model M Black Buckling Spring 104 Key USB Keyboard\n2. Unicomp Buckling Spring ANSI Color Key Sets - Red\n3. Unicomp Model M PC 122 Black 5250 Buckling Spring USB Keyboard\n4. Vinos tintos y blancos - Los mejores vinos de Castillo de Monjardín DO Navarra - Lote 6 bot.\n5. CHILEAF Bike Speedometer with Bike Light Set, Bicycle Headlight Taillight with Horn, USB Rechargeable Bike Front Light, Waterproof Bike Odometer and Tail Light\nYou should output a permutation of 1 to 5. There should be a comma separating two numbers. Each product and its number should appear only once in the output. Only respond with the ranking results.\nOutput: "},
    ],
)

In [ ]:
completion.choices[0].message.content

'1, 3, 2, 5, 4'

In [ ]:
import openai

from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import collections


# Define prediction function
def predict(params):
    query = params
    # Retry setup
    retry_count = 100
    retry_interval = 1
    for _ in range(retry_count):
        try:
            YOUR_KEY = ""
            YOUR_BASE_URL = "https://one-api.qkvlab.com/v1"
            YOUR_MODEL = "claude-3-opus-20240229"
            client = OpenAI(api_key=YOUR_KEY)
            client.base_url = YOUR_BASE_URL

            completion = client.chat.completions.create(
                model=YOUR_MODEL,
                messages=[
                    {"role": "system", "content": "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions."},
                    {"role": "user", "content": query},
                ],
            )
            # Extract GPT's response content
            msg = completion.choices[0].message.content
            return query, msg

        except openai.error.RateLimitError as e:
            print("Exceeded OpenAI API call frequency:", e)
            print('Retrying....')
            retry_count += 1
            retry_interval *= 2  # Exponential backoff
            time.sleep(retry_interval)

        except TimeoutError:
            print("Task execution timed out:", query)
            print('Retrying....')
            retry_count += 1
            retry_interval *= 2  # Exponential backoff
            time.sleep(retry_interval)

        except Exception as e:
            print("Error during task execution:", e)
            print('Retrying....')
            retry_count += 1
            retry_interval *= 2  # Exponential backoff
            time.sleep(retry_interval)

    return query, 'API request failed'


def main(data_list):
    start_time = time.time()
    input_data = data_list
    output_data = []
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(predict, query) for query in input_data]
        query2res = collections.defaultdict(int)
        for job in as_completed(futures):
            query, res = job.result(timeout=None)
            query2res[query] = res
            time.sleep(1)  # To avoid exceeding the OpenAI API rate limit

    end_time = time.time()
    total_run_time = round(end_time - start_time, 3)
    print('Total_run_time: {} s'.format(total_run_time))
    print(query2res)

    import pandas as pd
    df = pd.DataFrame({'input_field': list(query2res.keys()), 'output_field': list(query2res.values())})
    return df

predict_df = main(query_pair_formatted_statements_list)

In [ ]:
predict_df.to_csv('query_pair_labeled.csv', index=False)

In [ ]:
predict_df

,query,infer_result
0,Which of the following statements best describ...,2
1,Which of the following statements best describ...,2
2,Which of the following statements best describ...,2
3,Which of the following statements best describ...,3
4,Which of the following statements best describ...,1


In [ ]:
import openai
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Retry decorator with exponential backoff and stop condition
@retry(stop=stop_after_attempt(8), wait=wait_exponential(multiplier=1, max=60))
def predict(query):
    YOUR_KEY = ""
    YOUR_BASE_URL = "https://one-api.qkvlab.com/v1"
    YOUR_MODEL = "claude-3-opus-20240229"
    client = OpenAI(api_key=YOUR_KEY)
    client.base_url = YOUR_BASE_URL

    completion = client.chat.completions.create(
        model=YOUR_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions."},
            {"role": "user", "content": query},
        ],
    )
    # Ensure we are returning both the query and the response content
    return query, completion.choices[0].message.content

def main(data_list):
    results = []
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(predict, data): data for data in data_list}
        for future in as_completed(futures):
            query, response = future.result()
            results.append((query, response))
            # Save intermediate results to avoid data loss, now saving both query and response
            pd.DataFrame(results, columns=['Query', 'Response']).to_csv('intermediate_results.csv', index=False)
    return results

results = main(query_pair_formatted_statements_list)

In [ ]:
pd.DataFrame(results, columns=['Query', 'Response']).to_csv('query_pair_labeled.csv.csv', index=False)

In [ ]:
x = pd.DataFrame(results, columns=['Query', 'Response'])
x

,Query,Response
0,Which of the following statements best describ...,1
1,Which of the following statements best describ...,2
2,Which of the following statements best describ...,2
3,Which of the following statements best describ...,2
4,Which of the following statements best describ...,2
...,...,...
35995,Which of the following statements best describ...,2
35996,Which of the following statements best describ...,2
35997,Which of the following statements best describ...,2
35998,Which of the following statements best describ...,2


In [ ]:
retrieval_formatted_messages = pd.read_csv('retrieval_formatted_messages.csv')
retrieval_formatted_messages_list = retrieval_formatted_messages['formatted_message'].tolist()

In [ ]:
import openai
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Retry decorator with exponential backoff and stop condition
@retry(stop=stop_after_attempt(8), wait=wait_exponential(multiplier=1, max=60))
def predict(query):
    YOUR_KEY = ""
    YOUR_BASE_URL = "https://one-api.qkvlab.com/v1"
    YOUR_MODEL = "claude-3-opus-20240229"
    client = OpenAI(api_key=YOUR_KEY)
    client.base_url = YOUR_BASE_URL

    completion = client.chat.completions.create(
        model=YOUR_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions."},
            {"role": "user", "content": query},
        ],
    )
    # Ensure we are returning both the query and the response content
    return query, completion.choices[0].message.content

def main2(data_list):
    results = []  # Temporary storage for the current batch
    all_results = []  # Global list for storing all results

    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(predict, data): data for data in data_list}
        for future in as_completed(futures):
            query, response = future.result()
            result = (query, response)
            results.append(result)
            all_results.append(result)  # Store each result in the global list as well

            # Write to CSV and reset the temporary list when it reaches 500 results
            if len(results) >= 500:
                pd.DataFrame(results, columns=['Query', 'Response']).to_csv('intermediate_results2.csv', mode='a', header=False, index=False)
                results = []  # Reset the temporary list

    # Process the last batch of results that are less than 500
    if results:
        pd.DataFrame(results, columns=['Query', 'Response']).to_csv('intermediate_results2.csv', mode='a', header=False, index=False)

    return all_results  # Return the global list containing all results

all_results = main2(retrieval_formatted_messages_list)

In [ ]:
pd.DataFrame(all_results, columns=['Query', 'Response']).to_csv('retrieval_pair_labeled.csv', index=False)

In [ ]:
y = pd.DataFrame(all_results, columns=['Query', 'Response'])
y

,Query,Response
0,A user on an online shopping website has just ...,"1, 5, 7"
1,A user on an online shopping website has just ...,"3, 7, 13"
2,A user on an online shopping website has just ...,"1, 9, 10"
3,A user on an online shopping website has just ...,"1, 7, 15"
4,A user on an online shopping website has just ...,"2,3,4"
...,...,...
37995,You are a helpful shop assistant. A user would...,"2, 3, 8"
37996,You are a helpful shop assistant. A user would...,"1, 3, 9"
37997,You are a helpful shop assistant. A user would...,"2,3,5"
37998,You are a helpful shop assistant. A user would...,"2,3,5"


In [3]:
import pandas as pd
ranking_formatted_messages = pd.read_csv('ranking_formatted_statements.csv')
rankning_formatted_messages_list = ranking_formatted_messages['0'].tolist()
rankning_formatted_messages_list[:5]

["A user has made a query with keyword 'model m mechanical keyboard'. Given the following numbered list of 5 products, please rank the products according their relevance with the query. \nProduct List: \n1. Unicomp Two Tone Ultra Classic Model M Black Buckling Spring 104 Key USB Keyboard\n2. Unicomp Buckling Spring ANSI Color Key Sets - Red\n3. Unicomp Model M PC 122 Black 5250 Buckling Spring USB Keyboard\n4. Vinos tintos y blancos - Los mejores vinos de Castillo de Monjardín DO Navarra - Lote 6 bot.\n5. CHILEAF Bike Speedometer with Bike Light Set, Bicycle Headlight Taillight with Horn, USB Rechargeable Bike Front Light, Waterproof Bike Odometer and Tail Light\nYou should output a permutation of 1 to 5. There should be a comma separating two numbers. Each product and its number should appear only once in the output. Only respond with the ranking results.\nOutput: ",
 "A user has made a query with keyword 'thepriceof everything'. Given the following numbered list of 5 products, please

In [ ]:
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Retry decorator with exponential backoff and stop condition
@retry(stop=stop_after_attempt(8), wait=wait_exponential(multiplier=1, max=60))
def predict(query):
    YOUR_KEY = ""
    YOUR_BASE_URL = "https://one-api.qkvlab.com/v1"
    YOUR_MODEL = "claude-3-opus-20240229"

    client = OpenAI(api_key=YOUR_KEY)
    client.base_url = YOUR_BASE_URL

    completion = client.chat.completions.create(
        model=YOUR_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful online shopping assistant. Please answer the following question about online shopping and follow the given instructions."},
            {"role": "user", "content": query},
        ],
    )
    # Ensure we are returning both the query and the response content
    return query, completion.choices[0].message.content

def main3(data_list):
    results = []  # Temporary storage for the current batch
    all_results = []  # Global list for storing all results

    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(predict, data): data for data in data_list}
        for future in as_completed(futures):
            query, response = future.result()
            result = (query, response)
            results.append(result)
            all_results.append(result)  # Store each result in the global list as well

            # Write to CSV and reset the temporary list when it reaches 500 results
            if len(results) >= 500:
                pd.DataFrame(results, columns=['Query', 'Response']).to_csv('intermediate_results3.csv', mode='a', header=False, index=False)
                results = []  # Reset the temporary list

    # Process the last batch of results that are less than 500
    if results:
        pd.DataFrame(results, columns=['Query', 'Response']).to_csv('intermediate_results3.csv', mode='a', header=False, index=False)

    return all_results  # Return the global list containing all results

all_results2 = main3(rankning_formatted_messages_list[30500:])


In [ ]:
pd.DataFrame(all_results2, columns=['Query', 'Response']).to_csv('ranking_labeled.csv', index=False)

In [ ]:
z = pd.DataFrame(all_results2, columns=['Query', 'Response'])
z

In [ ]:
rankning_formatted_messages_list[:1][0]

"A user has made a query with keyword 'model m mechanical keyboard'. Given the following numbered list of 5 products, please rank the products according their relevance with the query. \nProduct List: \n1. Unicomp Two Tone Ultra Classic Model M Black Buckling Spring 104 Key USB Keyboard\n2. Unicomp Buckling Spring ANSI Color Key Sets - Red\n3. Unicomp Model M PC 122 Black 5250 Buckling Spring USB Keyboard\n4. Vinos tintos y blancos - Los mejores vinos de Castillo de Monjardín DO Navarra - Lote 6 bot.\n5. CHILEAF Bike Speedometer with Bike Light Set, Bicycle Headlight Taillight with Horn, USB Rechargeable Bike Front Light, Waterproof Bike Odometer and Tail Light\nYou should output a permutation of 1 to 5. There should be a comma separating two numbers. Each product and its number should appear only once in the output. Only respond with the ranking results.\nOutput: "